In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
sys.path.append(os.path.join(os.getcwd(), '../'))

from symlie.misc.utils_results import get_and_check_Ps
from symlie.misc.wandb import update_results_df, get_inspect_df
from symlie.model.setup import load_P_pred
from symlie.model.networks.linear import LinearP
from symlie.model.networks.implicit import LinearImplicit
from symlie.misc.viz import plot2d

from symlie.run import parse_options, main, process_args

In [2]:
args = parse_options(notebook=True)

args.seed = 1
args.tags = ['dev']
args.batch_size = 16
args.max_epochs = 100

args.y_low = 1
args.y_high = 3
args.noise_std = 0.5
args.grid_size = (1, 7)
args.eps_mult = [0.0, 0.0, 1.0, 0.0]
args.data_dir = '../data/sine1d'
args.net = 'TrainP'

args.n_train = 10000
args.lr = 0.001

args.criterion = 'mses'
args.lossweight_dg = 1.0
args.lossweight_o = 1.0

args.do_return = True
args.logger = None


process_args(args)

# Training
model, trainer, datamodule = main(args)

Seed set to 1


Initializing generator with size 7
Running without logging


/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [6]:
model.net.reset_parameters()

In [8]:
model.net.weight

tensor([[ 4.4272e-01,  7.4845e-01,  4.7724e-01, -1.0465e+00, -6.8257e-01,
          4.2569e-01,  5.1128e-01],
        [ 4.1303e-01,  3.6186e-01,  9.7838e-01,  1.4149e+00, -1.7569e+00,
          3.1415e-01, -9.9642e-01],
        [-4.6128e-01,  3.5305e-01, -1.1334e+00,  1.4962e+00,  4.3828e-01,
          8.3952e-01,  1.8825e+00],
        [-4.1530e-02, -2.3987e+00,  9.7604e-01, -7.2906e-01, -8.5534e-01,
          9.2019e-02, -4.1737e-01],
        [ 6.5992e-01, -5.2734e-01,  3.3331e-01, -7.8322e-01, -8.3364e-01,
          7.0028e-01, -1.7832e+00],
        [ 6.4986e-01,  1.0964e-05, -9.4343e-01, -2.5621e+00, -9.0340e-01,
         -1.1404e+00,  8.4726e-01],
        [ 9.2925e-01, -1.1099e+00,  2.9609e-01,  9.7363e-01, -6.8629e-02,
          4.6452e-01,  1.3876e+00]])

In [30]:
trainer.fit(model, datamodule)

<bound method Module.parameters of TransformationLearner(
  (net): LinearP()
  (generator): MLP(
    (0): Linear(in_features=11, out_features=7, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=7, out_features=7, bias=True)
    (4): Dropout(p=0.0, inplace=False)
  )
)>
Epoch 99: 100%|██████████| 7/7 [00:00<00:00, 45.51it/s, val_loss_o=0.228, val_loss_dg=0.874, val_loss=1.100, train_loss_o=0.139, train_loss_dg=0.828, train_loss=0.967] 


In [14]:
optimizer = model.configure_optimizers()

<bound method Module.parameters of TransformationLearner(
  (net): LinearP()
  (generator): MLP(
    (0): Linear(in_features=11, out_features=7, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=7, out_features=7, bias=True)
    (4): Dropout(p=0.0, inplace=False)
  )
)>


In [15]:
for name, param in model.named_parameters():
    print(name, param.shape)

net.P torch.Size([49, 49])
generator.0.weight torch.Size([7, 11])
generator.0.bias torch.Size([7])
generator.3.weight torch.Size([7, 7])
generator.3.bias torch.Size([7])


In [16]:
model.net.P.data

tensor([[-1.5256, -0.7502, -0.6540,  ...,  1.5748, -0.6298, -0.9274],
        [ 0.5451,  0.0663, -0.4370,  ...,  1.9907,  0.6614,  1.1899],
        [ 0.8165, -0.9135,  1.3851,  ..., -1.3633, -0.9832,  1.5113],
        ...,
        [ 2.3309, -1.0011, -1.0291,  ..., -1.1739,  1.0656, -0.3635],
        [ 0.0316,  1.0053, -0.5179,  ...,  1.4764, -0.2898,  0.2444],
        [ 0.3377, -0.7036, -0.0715,  ..., -0.0770, -0.5914,  1.2128]])

In [17]:
model.generator[0].weight.data

tensor([[ 0.0327, -0.2223,  0.1858,  0.1893, -0.0565, -0.2390,  0.2238, -0.2467,
         -0.1583,  0.1799, -0.1498],
        [ 0.0574,  0.2478, -0.0381, -0.0682,  0.2795,  0.0515,  0.1052, -0.0991,
         -0.2601,  0.0472, -0.1782],
        [-0.1559, -0.2473,  0.2583, -0.2024,  0.0872,  0.0620,  0.2082,  0.1647,
         -0.2111,  0.1337, -0.1532],
        [ 0.0111, -0.1092, -0.2581,  0.0902, -0.2012,  0.2950,  0.1215, -0.1043,
         -0.1766, -0.2779, -0.1520],
        [-0.1967,  0.0324,  0.0722,  0.2615,  0.0405, -0.0612, -0.1344, -0.0424,
          0.1594, -0.2907, -0.2761],
        [ 0.1190, -0.0930, -0.0892, -0.1943, -0.2008, -0.2258, -0.1835, -0.2757,
         -0.1417,  0.2419, -0.1668],
        [ 0.2292, -0.2538,  0.0350, -0.0250, -0.2277,  0.0073,  0.2511, -0.1362,
          0.2769, -0.0317, -0.0953]])

In [18]:
out = model.forward(batch)

out_o, out_dg = out
(lossweight_o, criterion_o), (lossweight_dg, criterion_dg) = model.criterion

loss_o = criterion_o(*out_o)
loss_dg = criterion_dg(*out_dg)
loss = lossweight_o*loss_o + lossweight_dg*loss_dg

loss.backward()
optimizer.step()

In [21]:
def mymodel(batch)
    out = model(batch)

    (out_a_prime, out_b_prime), (dg_x, dg_out) = out

    loss_o



True

/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/pytorch_lightning/core/module.py:420: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


torch.Size([])

In [27]:
# make_dot(model.step(batch, mode = 'val')[0], params=dict(model.named_parameters()))

In [26]:
x_ = torch.randn(16, 7)
e_ = torch.randn(4)

torch.cat([x_, e_.repeat(16, 1)], dim=1).shape

torch.Size([16, 11])

In [19]:
model.generator[0].weight.data

tensor([[ 0.0400, -0.2796,  0.2340,  0.2383, -0.0718, -0.3006,  0.2796],
        [-0.3082, -0.1975,  0.2265, -0.1868,  0.0710,  0.3097, -0.0488],
        [-0.0865,  0.3514,  0.0656,  0.1308, -0.1232, -0.3271,  0.0582],
        [-0.2224, -0.1965, -0.3110,  0.3228, -0.2547,  0.1103,  0.0788],
        [ 0.2621,  0.2074, -0.2656,  0.1665, -0.1930,  0.0149, -0.1379],
        [-0.3226,  0.1141, -0.2532,  0.3688,  0.1513, -0.1297, -0.2203],
        [-0.3474, -0.1896, -0.2456,  0.0416,  0.0915,  0.3268,  0.0498]])

In [21]:
model.net.P.data

tensor([[-1.5236, -0.7520, -0.6521,  ...,  1.5730, -0.6304, -0.9262],
        [ 0.5471,  0.0658, -0.4388,  ...,  1.9887,  0.6597,  1.1919],
        [ 0.8145, -0.9145,  1.3870,  ..., -1.3615, -0.9850,  1.5093],
        ...,
        [ 2.3291, -1.0006, -1.0295,  ..., -1.1750,  1.0637, -0.3655],
        [ 0.0296,  1.0073, -0.5198,  ...,  1.4745, -0.2901,  0.2464],
        [ 0.3367, -0.7055, -0.0719,  ..., -0.0763, -0.5894,  1.2148]])

In [22]:
model.generator[0].weight.data

tensor([[ 0.0406, -0.2789,  0.2343,  0.2377, -0.0726, -0.3006,  0.2787],
        [-0.3073, -0.1967,  0.2273, -0.1870,  0.0701,  0.3087, -0.0497],
        [-0.0869,  0.3523,  0.0663,  0.1301, -0.1235, -0.3264,  0.0586],
        [-0.2214, -0.1974, -0.3107,  0.3218, -0.2557,  0.1113,  0.0796],
        [ 0.2630,  0.2084, -0.2666,  0.1658, -0.1940,  0.0158, -0.1372],
        [-0.3216,  0.1151, -0.2542,  0.3679,  0.1504, -0.1288, -0.2195],
        [-0.3466, -0.1888, -0.2459,  0.0418,  0.0909,  0.3259,  0.0488]])